# Day 3 
## Part 1
These are escalating quickly this year.

In [1]:
def parse_data(s):
    return s.strip().splitlines()

test_data = parse_data("""467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..""")

test_data

['467..114..',
 '...*......',
 '..35..633.',
 '......#...',
 '617*......',
 '.....+.58.',
 '..592.....',
 '......755.',
 '...$.*....',
 '.664.598..']

In [2]:
def get_numbers(line):
    """Return the numbers and columns of where
    they start within a line"""
    i = 0
    while i < len(line):
        c = i
        if line[i].isdigit():
            new_num = ""
            while i < len(line) and line[i].isdigit():
                new_num += line[i]
                i += 1
            yield (new_num, c)
        i += 1
        
[list(get_numbers(line)) for line in test_data]

[[('467', 0), ('114', 5)],
 [],
 [('35', 2), ('633', 6)],
 [],
 [('617', 0)],
 [('58', 7)],
 [('592', 2)],
 [('755', 6)],
 [],
 [('664', 1), ('598', 5)]]

In [3]:
def get_surrounding_symbols(schematic, row, column, length):
    """Get the valid symbols around a string."""
    def yield_if_valid(r, c):
        if (
            r >= 0
            and r < len(schematic)
            and c >= 0
            and c < len(schematic[0])
        ):
            s = schematic[r][c]
            if not s.isdigit() and s != ".":
                yield s
    
    for r in (row - 1, row + 1):
        for c in range(column - 1, column + length + 1):
            yield from yield_if_valid(r, c)
    yield from yield_if_valid(row, column - 1)
    yield from yield_if_valid(row, column + length)
    
            
def numbers_with_surrounding_symbols(schematic):
    """Return numbers with any surrounding symbols
    from the schematic."""
    for r, line in enumerate(schematic):
        for number, c in get_numbers(line):
            yield (int(number), set(get_surrounding_symbols(schematic, r, c, len(number))))
            
list(numbers_with_surrounding_symbols(test_data))

[(467, {'*'}),
 (114, set()),
 (35, {'*'}),
 (633, {'#'}),
 (617, {'*'}),
 (58, set()),
 (592, {'+'}),
 (755, {'*'}),
 (664, {'$'}),
 (598, {'*'})]

In [4]:
def part_1(data):
    return sum(
        n 
        for n, symbols in numbers_with_surrounding_symbols(data)
        if symbols
    )

assert part_1(test_data) == 4361

In [5]:
data = parse_data(open("input").read())

part_1(data)

531932

## Part 2
Amend the `get_surrounding_symbols` function to yield coordinates of gears and solve from there.

In [6]:
from collections import defaultdict

def get_surrounding_gears(schematic, row, column, length):
    """Get the valid symbols around a string."""
    def yield_if_valid(r, c):
        if (
            r >= 0
            and r < len(schematic)
            and c >= 0
            and c < len(schematic[0])
        ):
            if schematic[r][c] == "*":
                yield (r, c)
    
    for r in (row - 1, row + 1):
        for c in range(column - 1, column + length + 1):
            yield from yield_if_valid(r, c)
    yield from yield_if_valid(row, column - 1)
    yield from yield_if_valid(row, column + length)
    
            
def gears_with_surrounding_numbers(schematic):
    """Return numbers with any surrounding symbols
    from the schematic."""
    gears = defaultdict(list)
    for r, line in enumerate(schematic):
        for number, c in get_numbers(line):
            for gear_r, gear_c in get_surrounding_gears(schematic, r, c, len(number)):
                gears[(gear_r, gear_c)].append(int(number))
    return gears

gears_with_surrounding_numbers(test_data)

defaultdict(list, {(1, 3): [467, 35], (4, 3): [617], (8, 5): [755, 598]})

In [7]:
import math

def part_2(data):
    return sum(
        math.prod(numbers)
        for numbers in gears_with_surrounding_numbers(data).values()
        if len(numbers) == 2
    )

assert part_2(test_data) == 467835

In [8]:
part_2(data)

73646890